In [1]:
%pip install google-adk

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()   # loads .env

GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")

if not GOOGLE_API_KEY:
    raise ValueError("❌ GEMINI_API_KEY not found. Add it to .env")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
print("✅ Gemini API key setup complete.")

✅ Gemini API key setup complete.


In [3]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [4]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

In [5]:
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings",  # The result of this agent will be stored in the session state with this key.
)

print("✅ research_agent created.")

✅ research_agent created.


In [6]:
summarizer_agent = Agent(
    name="SummarizerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary""",
)

print("✅ summarizer_agent created.")

✅ summarizer_agent created.


In [7]:
root_agent = Agent(
    name="ResearchCoordinator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""",

tools=[AgentTool(research_agent), AgentTool(summarizer_agent)],
)
print("✅ root_agent created.")

✅ root_agent created.


In [8]:
runner = InMemoryRunner(agent=root_agent)

# replace the prompt below with whatever you want to ask
prompt = "What are the latest advancements in quantum computing and what do they mean for AI?"

# # Run and capture the debug response object
response = await runner.run_debug(prompt)

# print("✅ run_debug completed — now inspect raw response candidates.")


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.



 ### Created new session: debug_session_id

User > What are the latest advancements in quantum computing and what do they mean for AI?


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.
Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


ResearchCoordinator > Recent advancements in quantum computing are poised to significantly impact Artificial Intelligence (AI) by enabling faster data processing, more efficient machine learning, and the ability to solve complex problems currently intractable for classical computers. This fusion, known as Quantum AI, promises unprecedented AI speeds and improved autonomous systems. Key developments include Google's Quantum Echoes algorithm demonstrating a significant speed advantage, and theoretical frameworks like the DQI algorithm for optimization. Major tech companies are heavily investing, with IBM targeting quantum advantage by 2026, and specialized hardware like China's optical quantum computing chip showing dramatic speedups for AI tasks. Progress in quantum error correction is also crucial for building stable, scalable quantum computers.


Now we will understand to work with Sequential WOrkflows

In [17]:
outline_agent = Agent(
    name="OutlineAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Create a blog outline for the given topic with:
    1. A catchy headline
    2. An introduction hook
    3. 3-5 main sections with 2-3 bullet points for each
    4. A concluding thought""",
    output_key="blog_outline",
)

print("✅ outline_agent created.")

✅ outline_agent created.


In [18]:
writrer_agent = Agent(
    name="WriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
    Write a brief, 200 to 300-word blog post with an engaging and informative tone.""",
    output_key="blog_post",
)

print("✅ writrer_agent created.")

✅ writrer_agent created.


In [19]:
editor_agent = Agent(
    name="EditorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
    Your task is to polish the text by fixing any grammatical errors, improving the flow and sentence structure, and enhancing overall clarity.""",
    output_key="final_blog",
)

print("✅ editor_agent created.")

✅ editor_agent created.


In [20]:
root_agent = SequentialAgent(
    name="BlogPipeline",
    sub_agents=[outline_agent,writrer_agent,editor_agent],
)

print("✅ root_agent with SequentialAgent created.")

✅ root_agent with SequentialAgent created.


In [21]:
runner = InMemoryRunner(agent=root_agent)

runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Write a blog post about the benefits of multi-agent systems for software developers"
)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.



 ### Created new session: debug_session_id

User > Write a blog post about the benefits of multi-agent systems for software developers


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


OutlineAgent > ## Outline:

**Headline:** Supercharge Your Code: How Multi-Agent Systems Are Revolutionizing Software Development

**Introduction Hook:** Imagine a team of highly specialized, collaborative "digital assistants" working tirelessly behind the scenes of your software projects. That's the power of multi-agent systems (MAS), and if you're a software developer, understanding them is about to become your next competitive edge.

**Main Sections:**

**1. What Exactly Are Multi-Agent Systems (MAS)?**

*   Brief, non-technical explanation of agents and how they interact.
*   Analogy to human teams or natural systems to make it relatable.
*   Key characteristics: autonomy, reactivity, proactivity, social ability.

**2. Boosting Development Efficiency and Productivity**

*   **Automated Task Distribution:** Agents can handle repetitive coding tasks, bug identification, and even initial test case generation, freeing up developers for complex problem-solving.
*   **Intelligent Code Co

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


WriterAgent > ## Supercharge Your Code: How Multi-Agent Systems Are Revolutionizing Software Development

Imagine a team of highly specialized, collaborative "digital assistants" working tirelessly behind the scenes of your software projects. That's the power of multi-agent systems (MAS), and if you're a software developer, understanding them is about to become your next competitive edge.

At its core, a MAS is a network of independent, intelligent agents that can communicate and cooperate to achieve common goals. Think of them like a swarm of highly skilled bees or a well-coordinated human development team, each agent with its own strengths and responsibilities. Key characteristics like autonomy, reactivity, proactivity, and social ability allow them to operate independently yet effectively together.

So, how does this translate to your daily grind? MAS can dramatically boost efficiency. Agents can automate repetitive coding tasks, identify bugs early, and even generate initial test c

Parallel Workflow

Parallel Workflows

In [37]:
tech_researcher = Agent(
    name="TechResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research the latest AI/ML trends. Include 3 key developments,
    the main companies involved, and the potential impact. Keep the report very concise (100 words).""",
    tools=[google_search],
    output_key="tech_research",
)

print("✅ tech_research_agent created.")



✅ tech_research_agent created.


In [38]:
health_researcher = Agent(
    name="HealthResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research recent medical breakthroughs. Include 3 significant advances,
their practical applications, and estimated timelines. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="health_research", 
)
print("✅ health_research_agent created.")

✅ health_research_agent created.


In [39]:
finance_researcher = Agent(
    name="FinanceResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research current fintech trends. Include 3 key trends,
their market implications, and the future outlook. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="finance_research",
)
print("✅ finance_researcher created.") 

✅ finance_researcher created.


In [40]:
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Combine these three research findings into a single executive summary:

    **Technology Trends:**
    {tech_research}
    
    **Health Breakthroughs:**
    {health_research}
    
    **Finance Innovations:**
    {finance_research}
    
    Your summary should highlight common themes, surprising connections, and the most important key takeaways from all three reports. The final summary should be around 200 words.""",
    output_key="executive_summary",
)

print("✅ aggregator_agent created.")

✅ aggregator_agent created.


In [41]:
parallel_research_agent = ParallelAgent(
    name="ParallelResearchAgent",
    sub_agents=[tech_researcher, health_researcher, finance_researcher],
)

root_agent = SequentialAgent(
    name="research_system",
    sub_agents=[parallel_research_agent, aggregator_agent],
)

print("✅ root_agent created.")

✅ root_agent created.


In [42]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Run the daily executive briefing on Tech, Health, and Finance"
)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.
Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.
Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.



 ### Created new session: debug_session_id

User > Run the daily executive briefing on Tech, Health, and Finance
HealthResearchAgent > Here's a concise briefing on recent breakthroughs in Tech, Health, and Finance:

**Health:** mRNA vaccine technology, initially proven with COVID-19 vaccines, is now being explored for numerous other diseases, with potential for faster development of future vaccines. AI is revolutionizing healthcare through faster sepsis detection and personalized treatment plans, projected to grow significantly. Robotic precision is also advancing, with applications in surgery and enhanced human capabilities.

**Technology:** Agentic AI, capable of handling end-to-end workflows, is a major trend. Neural interfaces merging biology and computing are moving from labs to practical applications, with an estimated market value of over $160 billion. Quantum computing continues to advance, promising future breakthroughs.

**Finance:** AI-driven financial advisors offering per

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


FinanceResearchAgent > **Tech Briefing:**
The tech industry is buzzing with AI advancements. OpenAI is piloting a new group chat feature for ChatGPT, while Mozilla develops an AI browsing feature for Firefox. Microsoft is building a massive AI "Super Factory" in Atlanta, and Google is investing significantly in data centers in Germany. Nvidia's market value has surged, though some legislative actions could curb its exports to China. The rise of AI is also impacting the job market, with AI implementation contributing to job cuts.

**Health Briefing:**
In healthcare, the FDA is considering removing "black box" warnings from some hormone replacement treatments for menopause symptoms, potentially encouraging more patients to seek treatment. There's a focus on rising syphilis infections in major cities, prompting new elimination plans. Conflicting advice on COVID-19 vaccines may lower vaccination rates, while libraries are increasingly serving as public health hubs. Breakthroughs in pig kid

Loop workflow

In [43]:
initial_writer_agent = Agent(
    name="InitialWriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Based on the user's prompt, write the first draft of a short story (around 100-150 words).
    Output only the story text, with no introduction or explanation.""",
    output_key="current_story",  # Stores the first draft in the state.
)

print("✅ initial_writer_agent created.")

✅ initial_writer_agent created.


In [44]:
critic_agent = Agent(
    name="CriticAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a constructive story critic. Review the story provided below.
    Story: {current_story}
    
    Evaluate the story's plot, characters, and pacing.
    - If the story is well-written and complete, you MUST respond with the exact phrase: "APPROVED"
    - Otherwise, provide 2-3 specific, actionable suggestions for improvement.""",
    output_key="critique",  # Stores the feedback in the state.
)

print("✅ critic_agent created.")

✅ critic_agent created.


In [45]:
def exit_loop():
    """Call this function ONLY when the critique is 'APPROVED', indicating the story is finished and no more changes are needed."""
    return {"status": "approved", "message": "Story approved. Exiting refinement loop."}


print("✅ exit_loop function created.")

✅ exit_loop function created.


In [46]:
refiner_agent = Agent(
    name="RefinerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a story refiner. You have a story draft and critique.
    
    Story Draft: {current_story}
    Critique: {critique}
    
    Your task is to analyze the critique.
    - IF the critique is EXACTLY "APPROVED", you MUST call the `exit_loop` function and nothing else.
    - OTHERWISE, rewrite the story draft to fully incorporate the feedback from the critique.""",
    output_key="current_story",  # It overwrites the story with the new, refined version.
    tools=[
        FunctionTool(exit_loop)
    ],  # The tool is now correctly initialized with the function reference.
)

print("✅ refiner_agent created.")

✅ refiner_agent created.


In [47]:
story_refinement_loop = LoopAgent(
    name="StoryRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=2,  # Prevents infinite loops
)

# The root agent is a SequentialAgent that defines the overall workflow: Initial Write -> Refinement Loop.
root_agent = SequentialAgent(
    name="StoryPipeline",
    sub_agents=[initial_writer_agent, story_refinement_loop],
)

print("✅ Loop and Sequential Agents created.")

✅ Loop and Sequential Agents created.


In [48]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Write a short story about a lighthouse keeper who discovers a mysterious, glowing map"
)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.



 ### Created new session: debug_session_id

User > Write a short story about a lighthouse keeper who discovers a mysterious, glowing map


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


InitialWriterAgent > Elias traced the salt-worn grooves of the lighthouse wall, his knuckles brushing against the smooth, cool stone. For thirty years, the rhythmic sweep of the beam had been his only companion, save for the gulls' cries and the ocean's endless murmur. Tonight, however, a faint luminescence flickered from a crevice he’d never noticed before. He reached in, his calloused fingers closing around something papery.

He pulled it out. It was a map, but unlike any he’d ever seen. The parchment was impossibly thin, and inked lines of ethereal blue pulsed with a soft, inner light. Strange symbols, not of any language he knew, adorned it, and a single, winding path seemed to lead out from the very island he called home, out into the inky blackness of the sea. A chill, not from the sea spray, ran down his spine.


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


CriticAgent > This story fragment sets up an intriguing mystery, but needs further development to be a complete narrative.

Here are a few suggestions:

*   **Develop Elias's reaction and internal conflict:** While Elias feels a chill, his reaction is quite subdued given the extraordinary nature of his discovery. What are his immediate thoughts? Does he dismiss it as a trick of the light, or is he immediately captivated and perhaps a little frightened? Exploring his internal monologue and his past experiences (or lack thereof with anything unusual) will make him a more relatable and engaging character.
*   **Hint at the map's potential purpose or origin:** The map is beautiful and mysterious, but its purpose is entirely unknown. Is it a treasure map? A guide to a hidden world? A warning? Even a subtle clue – perhaps a symbol that vaguely resembles something Elias has seen before, or a whisper of sound accompanying the map – could significantly deepen the intrigue and provide a stronger

In [ ]:
z